In [79]:
import os
import sys
import numpy as np
import nibabel as nib
import matplotlib
import matplotlib.pyplot as plt
import scipy.io

# Ajouter dossier source
sys.path.append("src")
from help_functions_Transform import *

# === PARAMÈTRES ===
case_id = "s0011"
z_min, z_max = 201, 210
seg_dir = f"Data_set/{case_id}/segmentations"
ct_path = f"Data_set/{case_id}/ct.nii.gz"
output_seg_dir = "segmentation_slices"
merged_dir = "merged_masks"
os.makedirs(output_seg_dir, exist_ok=True)
os.makedirs(merged_dir, exist_ok=True)




In [80]:
# === FICHIERS SEGMENTÉS ===
fichiers = [f for f in os.listdir(seg_dir) if f.endswith(".nii.gz")]
ORGANS = { f.rsplit('.nii', 1)[0]: [f] for f in fichiers }

# === DÉTECTION DES ORGANES PRÉSENTS DANS LES SLICES ===
organs_presnt = organs_present_in_crop(seg_dir, z_min, z_max, ORGANS)
print(f"Organes présents entre Z={z_min} et Z={z_max} :")
for org in organs_presnt:
    print(" -", org)



Organes présents entre Z=201 et Z=210 :
 - iliopsoas_left
 - autochthon_right
 - rib_left_10
 - rib_left_12
 - aorta
 - stomach
 - gallbladder
 - duodenum
 - rib_right_12
 - inferior_vena_cava
 - costal_cartilages
 - small_bowel
 - colon
 - vertebrae_L2
 - kidney_left
 - rib_left_11
 - autochthon_left
 - pancreas
 - liver
 - vertebrae_L1
 - iliopsoas_right
 - spinal_cord
 - rib_right_11
 - kidney_right


In [81]:
# === CHARGER LES MASQUES DÉTECTÉS ===
masks = {}
for org in organs_presnt:
    filename = ORGANS[org][0]
    img = nib.load(os.path.join(seg_dir, filename))
    data = img.get_fdata()
    mask_crop = data[:, :, z_min:z_max+1]
    masks[org] = nib.Nifti1Image(mask_crop, affine=img.affine)

# === AJOUT DU SOFT TISSUE ===
mask3d = Get_soft_tissue_mask(case_id, organs_presnt, ORGANS)
soft_tissue = mask3d[0][:, :, z_min:z_max+1]
ref_affine = nib.load(os.path.join(seg_dir, ORGANS[organs_presnt[0]][0])).affine
masks['soft_tissue'] = nib.Nifti1Image(soft_tissue.astype(np.uint8), affine=ref_affine)

# === GROUPEMENT DES MASQUES PAR PRÉFIXE (soft → 'soft') ===
groups = {}
for org in masks:
    key = 'soft' if org == 'soft_tissue' else org[:4]
    groups.setdefault(key, []).append(org)

# === FUSION DES MASQUES PAR GROUPE ===
merged_masks = {}
for key, organ_list in groups.items():
    imgs = [masks[org].get_fdata() for org in organ_list]
    merged_data = np.zeros_like(imgs[0], dtype=bool)
    for arr in imgs:
        merged_data |= (arr > 0)
    affine = masks[organ_list[0]].affine
    merged_masks[key] = nib.Nifti1Image(merged_data.astype(np.uint8), affine=affine)
    nib.save(merged_masks[key], os.path.join(merged_dir, f"{key}_mask_z{z_min}_{z_max}.nii.gz"))
    print(f"Groupe '{key}' fusionné à partir de {len(organ_list)} organes.")

print(f"\n✅ {len(merged_masks)} groupes fusionnés enregistrés dans '{merged_dir}'")



Groupe 'ilio' fusionné à partir de 2 organes.
Groupe 'auto' fusionné à partir de 2 organes.
Groupe 'rib_' fusionné à partir de 5 organes.
Groupe 'aort' fusionné à partir de 1 organes.
Groupe 'stom' fusionné à partir de 1 organes.
Groupe 'gall' fusionné à partir de 1 organes.
Groupe 'duod' fusionné à partir de 1 organes.
Groupe 'infe' fusionné à partir de 1 organes.
Groupe 'cost' fusionné à partir de 1 organes.
Groupe 'smal' fusionné à partir de 1 organes.
Groupe 'colo' fusionné à partir de 1 organes.
Groupe 'vert' fusionné à partir de 2 organes.
Groupe 'kidn' fusionné à partir de 2 organes.
Groupe 'panc' fusionné à partir de 1 organes.
Groupe 'live' fusionné à partir de 1 organes.
Groupe 'spin' fusionné à partir de 1 organes.
Groupe 'soft' fusionné à partir de 1 organes.

✅ 17 groupes fusionnés enregistrés dans 'merged_masks'


In [82]:
# === ATTRIBUTION DES COULEURS PAR GROUPE (tab20) ===
keys = list(merged_masks.keys())
cmap = matplotlib.colormaps.get_cmap('tab20')
group_colors = { key: cmap(i / len(keys)) for i, key in enumerate(keys) }

# === AFFICHAGE ET SAUVEGARDE DES IMAGES COLORÉES PAR SLICE ===
data0 = merged_masks[keys[0]].get_fdata()
h, w, d = data0.shape
for slice_idx in range(z_min, z_max+1):
    slice_local = slice_idx - z_min
    seg_img = np.zeros((h, w, 4), dtype=float)

    for key, img in merged_masks.items():
        mask_data = img.get_fdata()
        slice_mask = mask_data[:, :, slice_local] > 0
        color = group_colors[key]
        for c in range(4):
            seg_img[..., c][slice_mask] = color[c]

    fig, ax = plt.subplots(figsize=(5, 5))
    ax.imshow(seg_img)
    ax.axis('off')
    out_path = os.path.join(output_seg_dir, f"segmented_slice_{slice_idx}.png")
    fig.savefig(out_path, bbox_inches='tight', pad_inches=0)
    plt.close(fig)

print(f"✅ Images segmentées enregistrées dans '{output_seg_dir}' pour les slices {z_min} à {z_max}")



✅ Images segmentées enregistrées dans 'segmentation_slices' pour les slices 201 à 210


In [83]:
# === SAUVEGARDE DES COULEURS ===
group_names = list(group_colors.keys())
rgb = np.array([group_colors[k][:3] for k in group_names])
scipy.io.savemat('group_colors.mat', {
    'group_names': group_names,
    'rgb_colors': rgb
})
print("✅ Couleurs sauvegardées dans 'group_colors.mat'")

✅ Couleurs sauvegardées dans 'group_colors.mat'
